Cílem supercvičení je vyzkoušet si:
* jak dostat data do Hadoopu
* různé způsoby uložení dat v Hive (partitioning, formát, komprese)
* SQL dotazování nad velkými daty

## Nahrání dat do Hadoopu
Budeme pracovat se záznamy o průměrných teplotách na meteorologických stanicích v USA. Soubor s daty je uložen na lokálním FS:  
`/home/pascepet/fel_bigdata/data/teplota-usa.zip`  
Potřebujeme tato data dostat na Hadoop (tj. na HDFS). Tedy:  
* zkopírujte data do svého uživatelského adresáře (doporučuji si na to vytvořit podadresář);
* rozbalte;
* prohlédněte si několik řádků rozbalených dat a zjistěte počet řádků celkem (a zkuste si odpovědět na otázku, proč je dobré to udělat);
* vytvořte na HDFS ve svém uživatelském adresáři vhodný podadresář (stejná otázka jako výše &ndash; proč?);
* rozbalené soubory nakopírujte na HDFS do nově vytvořeného adresáře.

## Spuštění Hive konzoly
Spustíme Hive z příkazové řádky.

`beeline -u "jdbc:hive2://hador-c1.ics.muni.cz:10000/default;principal=hive/hador-c1.ics.muni.cz@ICS.MUNI.CZ"`

Zjistěte, zda existuje vaše databáze (název = vaše uživatelské jméno, zakládali jsme je na supercvičení Linux/SQL/Python). Pokud dosud neexistuje, vytvořte ji.  

**Přepněte na svou databázi** (`USE databaze_name`).

## 1. Vstupní data jako externí tabulka
1.1. Ze souborů nahraných v předchozím kroku na HDFS vytvořte externí tabulku *teplota_ext*. (Externí tabulka používá strukturu dat tak, jak jsou, v tomto kroku se žádné změny formátu apod. neprovádějí.)
  * CSV (text) formát
  * oddělovač polí je znak ","
  * oddělovač záznamů je znak ukončení řádky
  * první řádek obsahuje hlavičky a přeskočí se
  * soubor obsahuje tato pole:

| Pole        | Typ       | Popis                                      |
|-------------|-----------|--------------------------------------------|
| stanice     | string    | kód stanice                                |
| mesic       | int       | číslo měsíce                               |
| den         | int       | číslo dne v měsíci                         |
| hodina      | int       | číslo hodiny (1&ndash;24)                  |
| teplota     | int       | průměrná teplota, round(10&times;&deg;F)   |
| flag        | string    | kód úplnosti dat                           |
| latitude    | double    | GPS zeměpisná šířka (záporné = jižní)      |
| longitude   | double    | GPS zeměpisná délka (záporné = západní)    |
| vyska       | double    | nadmořská výška v metrech                  |
| stat        | string    | kód státu (vč. zámořských území)           |
| nazev       | string    | název stanice                              |

1.2. S externí tabulkou lze provádět SQL dotazy. Udělejte kontrolu:
  * výpis několika řádků tabulky *teplota_ext* a porovnání se vstupními daty;
  * celkový počet řádků a porovnání se vstupními daty (neměl by být přesně stejný &ndash; proč?);
  * počet řádků s hodnotami NULL u pole *teplota* (měl by být jen malá část z celkového počtu řádků).

## 2. Převod do optimalizované tabulky

2.1. Vytvořte si prázdnou interní (managed) tabulku *teplota*, ve které budou data uložena ve vhodnějším formátu a zkomprimovaná:
  * formát parquet
  * komprese snappy (je nutno zadat velkými písmeny SNAPPY)
  * teplotu budeme v této tabulce ukládat ve stupních Celsia, bude to tedy desetinné číslo

2.2. Vložte do tabulky *teplota* data z tabulky *teplota_ext*:
  * přepočtěte správně teplotu z 10&times;&deg;F na &deg;C;
  * ostatní pole převeďte beze změny.

2.3. Udělejte kontrolu tabulky *teplota*:
  * Vypište si několik řádků.
  * Zjistěte počet záznamů v tabulce *teplota* a porovnejte s počtem záznamů v tabulce *teplota_ext*.

2.4. Tabulka *teplota* je interní, a tedy jejím vlastníkem je Hive.
  * Najděte ji na HDFS pod `/user/hive/warehouse/jmeno_vasi_databaze.db` a zjistěte její velikost (počet MB).
  * Porovnejte velikost s velikostí externí tabulky (vámi nahraná data na HDFS, viz výše).

2.5. Zrušte v Hive externí tabulku *teplota_ext* (příkaz DROP TABLE). Prověřte, že tabulka sice již ve vaší databázi není, ale data na HDFS stále jsou.

## 3. Tabulka s partitions
3.1. Vytvořte si prázdnou interní (managed) tabulku *teplota_part*, která bude stejná jako tabulka *teplota* (tj. pole, jejich typy, formát, komprese), ale bude mít navíc partitioning podle měsíce. (Pozor na pořadí polí!)

3.2. Do tabulky *teplota_part* zkopírujte data z tabulky *teplota*, při kopírování vytvořte dynamický partitioning podle měsíce. Dynamický partitioning je potřeba předem povolit pomocí příkazů:  
```
set hive.exec.dynamic.partition=true;
set hive.exec.dynamic.partition.mode=nonstrict;
```

3.3. Najděte tabulku *teplota_part* na HDFS pod `/user/hive/warehouse/jmeno_vasi_databaze.db` a prohlédněte si, jak je partitioning realizován.

## 4. Dotazování nad Hive
Budeme pracovat s tabulkou *teplota*.

4.1. Kolik unikátních stanic je v datech? *(457)*

4.2. Která stanice je nejsevernější? *(USW00027502, BARROW POST ROGERS AP)*

4.3. Který stát má nejvíc unikátních stanic? *(TX)*

4.4. Kolik hodinových údajů celkem je na některé ze stanic v severní Dakotě (ND) nižších než &minus;10&deg;C? *(8&nbsp;446)*

4.5. Který stát má nejvyšší celkovou průměrnou teplotu na svých stanicích za letní měsíce (6, 7, 8)? *(MH, 28.1)*

4.6. Které všechny státy mají rozdíl zeměpisných šířek (longitude) mezi svou nejzápadnější a nejvýchodnější stanicí větší než 8 stupňů? *(AK, FM, MT, TX)*

4.7. Pro každou stanici s nadmořskou výškou nad 1&nbsp;500&nbsp;metrů zjistěte rozdíl mezi celkovou průměrnou teplotou stanice (za celý rok) a celkovou průměrnou teplotou státu, kam stanice patří.

| stanice     | rozdil             |
|:------------|-------------------:|
| USW00003103 | -9.732538702083291 |
| USW00023225 | -4.890214469033042 |
| ...         | ...                |
